In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import random

In [2]:
def obtener_info(soup):
    informacion = {}
    try:
        informacion['precio'] = soup.find("span", attrs = {"class": "_tyxjp1"}).text.replace('\xa0','')
        informacion['titulo_anuncio'] = browser.find_element(By.CLASS_NAME, '_1n81at5').text
        informacion['tipo_alojamineto'] = browser.find_element(By.CLASS_NAME, '_cv5qq4').text.split(':')[1].split('.')[0].strip()
        informacion['numero_personas'] = browser.find_element(By.CLASS_NAME, '_tqmy57').text.split("\n")[1].split("·")[0].strip()
        informacion['n_dormitorios'] = browser.find_element(By.CLASS_NAME, '_tqmy57').text.split("\n")[1].split("·")[1].strip()
        informacion['nunmero_camas'] = browser.find_element(By.CLASS_NAME, '_tqmy57').text.split("\n")[1].split("·")[2].strip()
        informacion['nunmero_banos'] = browser.find_element(By.CLASS_NAME, '_tqmy57').text.split("\n")[1].split("·")[3].strip()
        informacion['valoracion'] = browser.find_element(By.CLASS_NAME, '_klarpw').text.split(' ')[0]
        informacion['n_evaluaciones'] = browser.find_element(By.CLASS_NAME, '_klarpw').text.split(' ')[1].replace('·\n','')
    
    except Exception as e:
        print("Error:", e)
    return informacion

In [3]:
def avanzar_hasta_el_final(soup):
    sleep(1)
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
url_base = "https://www.airbnb.es/s/"
comunidades = ['valencia', 'galicia', 'asturias', 'cantabria', 'pais_vasco', 'navarra', 'la_rioja', 'aragon', 'cataluña', 'castilla_y_leon', 'madrid', 'comunidad_valenciana', 'castilla_la_mancha', 'murcia', 'extremadura', 'andalucia', 'ceuta', 'melilla', 'baleares' 'islas_canarias']

browser = webdriver.Chrome()

browser.maximize_window()

soup = BeautifulSoup(browser.page_source, "html.parser")

results = []
    
for comunidad in comunidades:
    url = url_base + comunidad + "/homes"
    browser.get(url)

    try:
        accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div[1]/div/div[4]/section/div[2]/div[2]')))
        accept_button.click()
    except:
        pass
    
    while True:
        try:
            soup = BeautifulSoup(browser.page_source, "html.parser")

            WebDriverWait(browser, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'c1l1h97y')))
            ad_elements = browser.find_elements(By.CLASS_NAME, 'c1l1h97y')

            for el in ad_elements:
            
                url = el.find_element(By.XPATH, './/a').get_attribute('href')
            
                results.append((comunidad, url))
            
            avanzar_hasta_el_final(soup)
            try:
                siguiente = browser.find_element(By.XPATH, '//a[@aria-label="Siguiente"]') 
                siguiente.click()
            except:
                break

        except:
            pass

In [40]:
random.shuffle(results)

In [41]:
pd.DataFrame(results, columns = ['comunidad', 'url']).to_csv("lugares.csv", index=False)

In [4]:
df_urls = pd.read_csv("lugares.csv")

In [5]:
INICIO = 0
FIN = 6000
df_urls = df_urls[INICIO:FIN]

In [51]:
# Empezar desde cero

numero_aleatorio = random.randint(1, 1000000000)


browser = webdriver.Chrome()

infos = []
for idx, row in df_urls.iterrows():
    url = row['url']
    browser.get(url)
    browser.maximize_window()
    sleep(2)
        
    try:
        cerrar_traduccion = browser.find_element(By.CLASS_NAME, '_1piuevz').click()
        accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div[1]/div/div[4]/section/div[2]/div[2]')))
        accept_button.click()
    except:
        pass
        
    response = requests.get
    soup = BeautifulSoup(browser.page_source, "html.parser")
    sleep(1)
    info = obtener_info(soup)
    info["comunidad"] = row['comunidad']
    info["url"] = url
    sleep(4)
    infos.append(info)
        
    info_df = pd.DataFrame(infos)
    info_df.to_csv(f"info_{numero_aleatorio}.csv")

In [ ]:
# Usar desde copia de seguridad

numero_aleatorio = random.randint(1, 1000000000)

info_df = pd.read_csv("info_397144036.csv")

browser = webdriver.Chrome()

for idx, row in df_urls.iterrows():
    url = row['url']
    if url in info_df.url.values:
        continue
    
    browser.get(url)
    browser.maximize_window()
    sleep(2)
        
    try:
        cerrar_traduccion = browser.find_element(By.CLASS_NAME, '_1piuevz').click()
        accept_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div/div/div[1]/div/div[4]/section/div[2]/div[2]')))
        accept_button.click()
    except:
        pass
        
    response = requests.get
    soup = BeautifulSoup(browser.page_source, "html.parser")
    sleep(1)
    info = obtener_info(soup)
    info["comunidad"] = row['comunidad']
    info["url"] = url
    sleep(4)
    info_df.loc[len(info_df), :] = info
    info_df.to_csv(f"info_{numero_aleatorio}.csv")

In [1]:
##############################################################################################################################################

In [ ]:
url = f"https://www.bankinter.com/blog/finanzas-personales/precio-vivienda-ciudades"

r = requests.get(url)

soup = bs(r.text,'html.parser')

tabla = soup.find('tbody')

datos = []

if tabla:
    filas = tabla.find_all('tr')
    
    for fila in filas:
        celdas = fila.find_all('td')
        if len(celdas) == 4:
            ciudad = celdas[0].text.strip()
            precio_m2 = celdas[1].text.strip()
            variacion_anual = celdas[2].text.strip()
            datos.append((ciudad, precio_m2, variacion_anual))
    
    df = pd.DataFrame(datos, columns=['Ciudad', 'Precio por m2', 'Variación Anual'])
    
    print(df)

In [ ]:
df.to_csv(f"preciom2.csv")